### Baseline Model

In [1]:
import pandas as pd

# Load the data with low_memory=False to avoid DtypeWarning
df = pd.read_csv('clean_data.csv', low_memory=False)

df = df.drop_duplicates(subset=['Epitope - Name'])

# Filter rows where 'Epitope - Object Type' is 'Linear peptide'
filtered_df = df[df['Epitope - Object Type'] == 'Linear peptide']

# Rename the columns
filtered_df.rename(columns={'Assay - Qualitative Measure': 'Label', 'Epitope - Name': 'Sequence'}, inplace=True)

# Ensure you're operating on a copy to avoid SettingWithCopyWarning
final_df = filtered_df[['Sequence', 'Label']].copy()

# Display the final DataFrame to verify
final_df

,Sequence,Label
0,PTRAETREERMERKRREKIE,1
1,ERKRR,1
3,AEEEEDDDMGFGLFD,1
4,ASAPTAAAAASGGAAAPAA,0
5,EEEDDDMGFGLFD,1
...,...,...
5498,NSLLNLEKTMVR,1
5499,DSTCPMVTAPCS,1
5500,GLYHSNASFRVP,1
5501,LTNPGLGSSPKA,1


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

# Check for GPU availability and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load ProtBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

# Move the model to the specified device (GPU if available)
model = BertModel.from_pretrained("Rostlab/prot_bert").to(device)
model.eval()  # Set the model to evaluation mode

# Preprocessing and encoding functions
def clean_sequence(seq):
    """ Remove extra information after '+' in the sequence """
    return seq.split('+')[0].strip()

def preprocess_sequence(seq):
    return ' '.join(seq)  # Ensure each amino acid is separated by a space

def encode_sequence(seq):
    encoded_input = tokenizer(seq, return_tensors='pt').to(device)  # Move encoded input to GPU
    with torch.no_grad():
        outputs = model(**encoded_input)
    return outputs.pooler_output


# Encode sequences with progress bar
df = final_df.copy()
#df['Label'] = df['Label'].apply(lambda x: 1 if x == 'Positive' else 0)

df['Sequence'] = df['Sequence'].apply(clean_sequence)  # Clean sequences

# Detect duplicates and inconsistencies
duplicates = df.duplicated(subset=['Sequence'], keep=False)
inconsistencies = df[duplicates].groupby('Sequence').nunique()['Label'] > 1
inconsistent_sequences = inconsistencies[inconsistencies].index

if not inconsistent_sequences.empty:
    print(f"Warning: Removing {len(inconsistent_sequences)} inconsistent sequences with different labels: {inconsistent_sequences.tolist()}")
    df = df[~df['Sequence'].isin(inconsistent_sequences)]

# Deduplicate entries after removing inconsistencies
df = df.drop_duplicates(subset=['Sequence'], keep='first')

# Encode sequences
df['Encoded'] = [preprocess_sequence(seq) for seq in df['Sequence']]
df

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,Sequence,Label,Encoded
0,PTRAETREERMERKRREKIE,1,P T R A E T R E E R M E R K R R E K I E
1,ERKRR,1,E R K R R
3,AEEEEDDDMGFGLFD,1,A E E E E D D D M G F G L F D
4,ASAPTAAAAASGGAAAPAA,0,A S A P T A A A A A S G G A A A P A A
5,EEEDDDMGFGLFD,1,E E E D D D M G F G L F D
...,...,...,...
5498,NSLLNLEKTMVR,1,N S L L N L E K T M V R
5499,DSTCPMVTAPCS,1,D S T C P M V T A P C S
5500,GLYHSNASFRVP,1,G L Y H S N A S F R V P
5501,LTNPGLGSSPKA,1,L T N P G L G S S P K A


In [3]:
# feature extraction
embeddings = torch.cat([encode_sequence(seq) for seq in tqdm(df['Encoded'], desc="Encoding Sequences")]).to(device)
labels = torch.tensor(df['Label'].tolist(),  dtype=torch.float).to(device)

Encoding Sequences: 100%|██████████| 4451/4451 [02:17<00:00, 32.41it/s]


In [4]:
embeddings[1]

tensor([-0.2717,  0.2866, -0.2588,  ...,  0.2734,  0.2558, -0.2791],
       device='cuda:0')

In [5]:
labels

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

In [6]:
# Define the neural network with added complexity
class SimpleNNplus(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNNplus, self).__init__()
        self.layer1 = nn.Linear(input_dim, 512)
        self.relu1 = nn.LeakyReLU()
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.layer2 = nn.Linear(512, 128)
        self.relu2 = nn.LeakyReLU()
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.layer3 = nn.Linear(128, 64)
        self.relu3 = nn.LeakyReLU()
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.layer4 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.batchnorm1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.batchnorm2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        x = self.batchnorm3(x)
        x = self.layer4(x)
        x = self.sigmoid(x)
        return x


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR


# Setup neural network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = embeddings.shape[1]
classifier = SimpleNNplus(input_dim)
classifier.to(device)

# Split data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42, stratify= labels.to('cpu'))
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Convert to dataloaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the optimizer

# Binary Cross Entropy Loss for binary classification
criterion = nn.BCELoss()
optimizer = optim.AdamW(classifier.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=25, gamma=0.1)  # Decreases the LR by a factor of 0.1 every 25 epochs

# Training loop
total_epochs = 1000
for epoch in tqdm(range(total_epochs), desc="Training Progress"):
    epoch_losses = []
    classifier.train()
    for inputs, batch_labels in train_loader:
        inputs, batch_labels = inputs.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = criterion(outputs.squeeze(), batch_labels)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    scheduler.step()  # Update the learning rate
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Average Loss: {sum(epoch_losses)/len(epoch_losses)}')

# Evaluation
classifier.eval()
with torch.no_grad():
    y_preds = []
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = classifier(inputs)
        y_preds.extend(outputs.squeeze().tolist())
    y_preds = torch.tensor(y_preds).to('cpu')

    y_preds_np = y_preds.round().numpy()
    y_test_np = y_test.to('cpu').numpy()
    accuracy = accuracy_score(y_test_np, y_preds_np)
    f1 = f1_score(y_test_np, y_preds_np)
    auc_roc = roc_auc_score(y_test_np, y_preds_np)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"AU-ROC: {auc_roc}")


Training Progress:   0%|          | 1/1000 [00:00<09:05,  1.83it/s]

Epoch 1, Average Loss: 0.6426975200218814


Training Progress:  10%|█         | 101/1000 [00:34<04:24,  3.40it/s]

Epoch 101, Average Loss: 0.517629226403577


Training Progress:  20%|██        | 201/1000 [01:05<04:01,  3.30it/s]

...

Epoch 801, Average Loss: 0.52019567388509


Training Progress:  90%|█████████ | 901/1000 [04:45<00:29,  3.38it/s]

Epoch 901, Average Loss: 0.5220147733177457


Training Progress: 100%|██████████| 1000/1000 [05:16<00:00,  3.16it/s]

Accuracy: 0.7654320987654321
F1 Score: 0.009478672985781991
AU-ROC: 0.5023809523809524


In [ ]:
# Split dataset into training and validation sets
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
batch_size = 16
# Create DataLoader for training set
train_sampler = torch.utils.data.SubsetRandomSampler (train_indices)
train_loader = DataLoader (dataset, batch_size-batch_size, sampler=train_sampler)
# Create DataLoader for validation set
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset, batch_size-batch_size, sampler=val
_sampler)

In [ ]:
import itertools

def generate_configurations(batch_sizes, epochs):
    # Use itertools.product to create all combinations of batch_sizes and epochs
    product = itertools.product(batch_sizes, epochs)
    # Convert each combination into a dictionary and return as a list
    configurations = [{'batch_size': b, 'epochs': e} for b, e in product]
    return configurations

# Example usage
batch_sizes = [8, 16, 32]
epochs = [50, 100, 500, 1000]
configs = generate_configurations(batch_sizes, epochs)

# Print configurations to verify
for config in configs:
    print(config)

{'batch_size': 8, 'epochs': 50}
{'batch_size': 8, 'epochs': 100}
{'batch_size': 8, 'epochs': 500}
{'batch_size': 8, 'epochs': 1000}
{'batch_size': 16, 'epochs': 50}
{'batch_size': 16, 'epochs': 100}
{'batch_size': 16, 'epochs': 500}
{'batch_size': 16, 'epochs': 1000}
{'batch_size': 32, 'epochs': 50}
{'batch_size': 32, 'epochs': 100}
{'batch_size': 32, 'epochs': 500}
{'batch_size': 32, 'epochs': 1000}


test 004

In [ ]:
3467/20399

0.1699593117309672

In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import itertools
from torch.optim.lr_scheduler import StepLR

def generate_configurations(batch_sizes, epochs):
    # Use itertools.product to create all combinations of batch_sizes and epochs
    product = itertools.product(batch_sizes, epochs)
    # Convert each combination into a dictionary and return as a list
    configurations = [{'batch_size': b, 'epochs': e} for b, e in product]
    return configurations

# Example usage
batch_sizes = [8, 16, 32, 64]
epochs = [50, 100, 500, 1000]
#batch_sizes = [8]
#epochs = [10]
configs = generate_configurations(batch_sizes, epochs)


def run_experiment(config, train_dataset, test_dataset, classifier, criterion, optimizer, scheduler):
    total_epochs = config['epochs']
    batch_size = config['batch_size']
    #train_loader.batch_size = batch_size  # Adjusting batch size for the DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    # Lists to store metrics
    epoch_losses = []
    train_accuracies = []

    for epoch in tqdm(range(total_epochs), desc=f"Training with batch size={batch_size}"):
        # Training
        classifier.train()
        total = 0
        correct = 0
        for inputs, batch_labels in train_loader:
            inputs, batch_labels = inputs.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = classifier(inputs)
            loss = criterion(outputs.squeeze(), batch_labels)
            loss.backward()
            optimizer.step()

            # Calculate training accuracy
            predicted = outputs.round().squeeze(-1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
            epoch_losses.append(loss.item())

        train_accuracy = correct / total
        train_accuracies.append(train_accuracy)
        # print(f"Training Accuracy: {train_accuracy:.2f}%")  # Print as a formatted percentage

        scheduler.step()  # Update learning rate

    # Evaluation
    classifier.eval()
    with torch.no_grad():
        y_preds = []
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = classifier(inputs)
            y_preds.extend(outputs.squeeze().tolist())

    y_preds = torch.tensor(y_preds).to('cpu')
    y_preds_np = y_preds.round().numpy()
    y_test_np = y_test.to('cpu').numpy()
    accuracy = accuracy_score(y_test_np, y_preds_np)
    f1 = f1_score(y_test_np, y_preds_np)
    roc_auc = roc_auc_score(y_test_np, y_preds_np)

    return {
        'batch_size': batch_size,
        'epochs': total_epochs,
        'train_accuracy': sum(train_accuracies) / len(train_accuracies),
        'final_accuracy': accuracy,
        'F1_score': f1,
        'roc_auc': roc_auc
    }


# Setup neural network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = embeddings.shape[1]
classifier = SimpleNNplus(input_dim)
classifier.to(device)

# Split data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42, stratify= labels.to('cpu'))
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Convert to dataloaders
train_dataset = TensorDataset(X_train, y_train)
#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
#test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Binary Cross Entropy Loss for binary classification
criterion = nn.BCELoss()
optimizer = optim.AdamW(classifier.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=25, gamma=0.1)  # Decreases the LR by a factor of 0.1 every 25 epochs

# Collect results from all configurations
results = []
for config in configs:
    result = run_experiment(config, train_dataset, test_dataset, classifier, criterion, optimizer, scheduler)
    results.append(result)

# Convert results to DataFrame and print
results_df = pd.DataFrame(results)
print(results_df)
#results_df.to_csv('experiment_results.csv', index=False)


Training with batch size=64: 100%|██████████| 1000/1000 [02:02<00:00,  8.16it/s]

    batch_size  epochs  train_accuracy  final_accuracy  F1_score   roc_auc
0            8      50        0.791820        0.757785  0.045455  0.506708
1            8     100        0.795212        0.755479  0.009346  0.498610
2            8     500        0.795212        0.755479  0.018519  0.500255
3            8    1000        0.795212        0.758939  0.018779  0.502532
4           16      50        0.795212        0.757785  0.009434  0.500128
5           16     100        0.795212        0.761246  0.018957  0.504049
6           16     500        0.795212        0.758939  0.000000  0.499241
7           16    1000        0.795212        0.756632  0.000000  0.497724
8           32      50        0.795212        0.757785  0.000000  0.498483
9           32     100        0.795212        0.757785  0.000000  0.498483
10          32     500        0.795212        0.758939  0.000000  0.499241
11          32    1000        0.795212        0.756632  0.000000  0.497724
12          64      50   

In [ ]:
results_df.to_csv('ProtBERT_experiment_results.csv', index=False)

In [ ]:
%pwd

'/gpfs/gibbs/project/cpsc477/cpsc477_ly349/DL_project/ProtBERT'

### Integrating Biological Information

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [8]:
df = pd.read_csv('clean_data.csv', low_memory=False)
df = df.drop_duplicates(subset=['Epitope - Name'])

In [9]:
# Rename the columns
df.rename(columns={'Assay - Qualitative Measure': 'Label', 'Epitope - Name': 'Sequence'}, inplace=True)

In [10]:
categorical_cols = ['Epitope - Source Molecule', 'Epitope - Source Organism', 'Host - Name', 'Host - Age',
                    '1st in vivo Process - Process Type', '1st in vivo Process - Disease',
                    '1st in vivo Process - Disease Stage', 'Assay - Method', 'Assay - Response measured',
                    'Assay Antibody - Antibody Source Material', 'Assay Antibody - Immunoglobulin Domain',
                    'Assay Antibody - Purification Status', 'Assay Antibody - Heavy chain isotype',
                    'Assay Antigen - Epitope Relation', 'Assay Antigen - Object Type',
                    'Assay Antigen - Source Organism', 'Assay Antigen - Species']

continuous_cols = ['Epitope - Starting Position', 'Epitope - Ending Position',
                   'Adoptive Transfer - Flag', 'Assay - Response Frequency (%)']

In [11]:
# Impute missing values for continuous columns
imputer = SimpleImputer(strategy='mean')
df[continuous_cols] = imputer.fit_transform(df[continuous_cols])

# One-hot encode categorical columns
encoder = OneHotEncoder(sparse=False)
encoded_categorical = encoder.fit_transform(df[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_cols))

# Standardize continuous columns
scaler = StandardScaler()
df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
# Check the number of rows in each dataset
print("Number of rows in embeddings tensor:", embeddings.shape[0])
print("Number of rows in encoded_categorical_df:", encoded_categorical_df.shape[0])
print("Number of rows in continuous features dataframe:", df[continuous_cols].shape[0])

Number of rows in embeddings tensor: 4451
Number of rows in encoded_categorical_df: 4451
Number of rows in continuous features dataframe: 4451


In [13]:
# Combine encoded categorical features, continuous features, and sequence embeddings
combined_features = torch.cat([embeddings, torch.tensor(encoded_categorical_df.values, dtype=torch.float).to(device),
                               torch.tensor(df[continuous_cols].values, dtype=torch.float).to(device)], dim=1)

In [14]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2, random_state=42)

# Convert to dataloaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
# Initialize the model
input_dim = combined_features.shape[1]
classifier = SimpleNNplus(input_dim)
classifier.to(device)

SimpleNNplus(
  (layer1): Linear(in_features=1371, out_features=512, bias=True)
  (relu1): LeakyReLU(negative_slope=0.01)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer2): Linear(in_features=512, out_features=128, bias=True)
  (relu2): LeakyReLU(negative_slope=0.01)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer3): Linear(in_features=128, out_features=64, bias=True)
  (relu3): LeakyReLU(negative_slope=0.01)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer4): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [16]:
# Define the optimizer and loss function
criterion = nn.BCELoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

In [17]:
# Training loop
total_epochs = 100
for epoch in tqdm(range(total_epochs), desc="Training Progress"):
    epoch_losses = []
    classifier.train()
    for inputs, batch_labels in train_loader:
        inputs, batch_labels = inputs.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = criterion(outputs.squeeze(), batch_labels)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    #scheduler.step()  # Update the learning rate
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Average Loss: {sum(epoch_losses)/len(epoch_losses)}')

Training Progress:   1%|          | 1/100 [00:00<00:43,  2.25it/s]

Epoch 1, Average Loss: 0.24428508853140687


Training Progress: 100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


In [18]:
# Evaluation
classifier.eval()
with torch.no_grad():
    y_preds = []
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = classifier(inputs)
        y_preds.extend(outputs.squeeze().tolist())
    y_preds = torch.tensor(y_preds).to('cpu')

    y_preds_np = y_preds.round().numpy()
    y_test_np = y_test.to('cpu').numpy()
    accuracy = accuracy_score(y_test_np, y_preds_np)
    f1 = f1_score(y_test_np, y_preds_np)
    auc_roc = roc_auc_score(y_test_np, y_preds_np)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"AU-ROC: {auc_roc}")

Accuracy: 0.9180695847362514
F1 Score: 0.8443496801705758
AU-ROC: 0.926641493601846
